## Python Imports

In [1]:
!pip install accelerate
!pip install transformers
!pip install sentence-transformers
!pip install datasets
# !pip -q install google-generativeai==0.3.0
# !pip -q install google-ai-generativelanguage==0.4.0
import accelerate
import torch
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import TensorDataset
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt

import os
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import get_linear_schedule_with_warmup
from datasets import Dataset, DatasetDict
from accelerate import Accelerator
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

In [2]:
# from google.colab import files
# import shutil

# !mkdir -p /content/dataset

# uploaded = files.upload()
# for filename in uploaded.keys():
#     shutil.move(filename, '/content/dataset/')

In [3]:
import pandas as pd
import requests

# URL for the annotations file
annotations_url = "https://raw.githubusercontent.com/Kushal-Chandani/NLP-Project/main/train%20data/EN/subtask-1-annotations.txt"

# Load the annotations file
df = pd.read_csv(annotations_url, sep="\t", header=None, names=['article_id', 'entity_mention', 'start_offset', 'end_offset', 'main_role', 'fine_grained_role1', 'fine_grained_role2'])

# Base URL for the raw documents
base_url = "https://raw.githubusercontent.com/Kushal-Chandani/NLP-Project/main/train%20data/EN/raw-documents/"

#delete last column
df = df.drop(columns=['fine_grained_role2'])

# Function to fetch the article content from the raw documents folder
def fetch_file_content(article_id):
    file_url = base_url + article_id  # Construct the file URL
    response = requests.get(file_url)
    if response.status_code == 200:
        return response.text  # Return the content if the file is found
    else:
        return f"File {article_id} not found"

df['article_content'] = df['article_id'].apply(fetch_file_content)

def extract_contextual_content(row):
    content = row['article_content']
    start = row['start_offset']
    end = row['end_offset']

    if isinstance(content, str) and len(content) >= end:
        # Determine the range for 'before' based on the start position
        sentence_start = max(0, start - 250)
        sentence_end = min(len(content), end + 250)
    return content[sentence_start:sentence_end]


df['contextual_content'] = df.apply(extract_contextual_content, axis=1)
df = df[df['contextual_content'].str.len() > 350]
df = df.reset_index()
df


index        article_id       entity_mention  start_offset  end_offset  \
0        0  EN_UA_103861.txt              Chinese           791         797   
1        1  EN_UA_103861.txt                China          1516        1520   
2        2  EN_UA_103861.txt                Hamas          2121        2125   
3        4  EN_UA_021270.txt               Yermak           667         672   
4        5  EN_UA_021270.txt             Zelensky           846         853   
..     ...               ...                  ...           ...         ...   
394    409  EN_UA_027787.txt    the United States           105         121   
395    410  EN_UA_027787.txt              Kremlin          1864        1870   
396    411  EN_UA_027787.txt      Dmitry Medvedev          1970        1984   
397    412  EN_UA_027787.txt  U.S. administration          2269        2287   
398    413  EN_UA_027787.txt    Ukrainian leaders          2290        2306   

       main_role fine_grained_role1  \
0     Antagonist                Spy   
1     Antagonist         Instigator   
2     Antagonist          Terrorist   
3     Antagonist        Incompetent   
4     Antagonist        Incompetent   
..           ...                ...   
394   Antagonist         Instigator   
395  Protagonist           Guardian   
396  Protagonist           Guardian   
397   Antagonist          Terrorist   
398   Antagonist          Terrorist   

                                       article_content  \
0    The World Needs Peacemaker Trump Again \n\n by...   
1    The World Needs Peacemaker Trump Again \n\n by...   
2    The World Needs Peacemaker Trump Again \n\n by...   
3    Ukraine's Fate Will Be Decided In Coming Year,...   
4    Ukraine's Fate Will Be Decided In Coming Year,...   
..                                                 ...   
394  Russia Says US Is Responsible for Deadly Ukrai...   
395  Russia Says US Is Responsible for Deadly Ukrai...   
396  Russia Says US Is Responsible for Deadly Ukrai...   
397  Russia Says US Is Responsible for Deadly Ukrai...   
398  Russia Says US Is Responsible for Deadly Ukrai...   

                                    contextual_content  
0    r in October. If this rate continues, this yea...  
1    l it completed its trek across many states. In...  
2    umber of injuries and deaths in the war neared...  
3    aff for the Office of the President of Ukraine...  
4    on in an address to the hawkish DC-based Hudso...  
..                                                 ...  
394  Russia Says US Is Responsible for Deadly Ukrai...  
395   injured, including 82 taken to hospitals. Twe...  
396  rs were being flown in from other parts of Rus...  
397  Council, described the incident as “a vile, de...  
398  e incident as “a vile, despicable act against ...  

[399 rows x 9 columns]

In [4]:
counts = df['main_role'].value_counts()
counts

main_role
Antagonist     254
Protagonist     99
Innocent        46
Name: count, dtype: int64

In [5]:
import google.generativeai as genai
from google.colab import userdata
import pandas as pd

from IPython.display import display
from IPython.display import Markdown

GOOGLE_AI_STUDIO = userdata.get('gggg')

genai.configure(api_key=GOOGLE_AI_STUDIO)

In [6]:
!pip -q install google-generativeai==0.3.0
!pip -q install google-ai-generativelanguage==0.4.0

# for m in genai.list_models():
#     if 'generateContent' in m.supported_generation_methods:
#         print(m.name)

# models = genai.GenerativeModel('gemini-1.5-flash-002')

In [7]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models = genai.GenerativeModel('gemini-1.5-flash-002')

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/learnlm-1.5-pro-experimental
models/gemini-exp-1114
models/gemini-exp-1121


In [8]:
all_entities = df['entity_mention']
all_entities[222]

'the West'

In [9]:
import pandas as pd
import time
from collections import Counter

# List of tuples for fine labels and counts
roles = [
    ("Protagonist", 103), ("Innocent", 47)
]

# Converting roles list to a dictionary for checking counts
roles_dict = dict(roles)

# To track generated counts for each fine_label
generated_counts = Counter()


dff = df.copy()
generated_sentences = []
generated_labels = []
generated_entities = []  # To store mentioned entities
generated_finelabels = []

for index, row in dff.iterrows():
    sentence = row['sentence']
    fine_label = row['fine_grained_role1']
    label = row['label']
    print(f"Processing index: {index}, label: {label}")

    if generated_counts[label] >= 200:
            continue
    if index > 232 and index < 400:
        if label in roles_dict:
            loop_count = 0
            while generated_counts[fine_label] < 200 and loop_count < 2:

                # Pick an unused entity name
                if loop_count == 0:
                    ent_val = 1
                elif loop_count == 1:
                    ent_val = 2
                elif loop_count == 2:
                    ent_val = 3
                elif loop_count == 3:
                    ent_val = 4

                entity = all_entities[index+ent_val]

                time.sleep(7)  # Delay to respect generation constraints

                response = models.generate_content([
                    f"Generate a sentence based on the context: '{sentence}' using the mentioned entity: '{entity}'. "
                    f"The sentence should contain the mentioned entity:'{entity}'. "
                    f"The sentence should reflect the '{label}' and '{fine_label}' role perspective with respect to the '{entity}'. Each sentence should be exactly 500 characters including the entity name. "
                    f"Do not start with a number or special character."
                ])

                if response.parts:
                    sen = response.text.strip()
                    generated_sentences.append(sen)
                    generated_labels.append(label)
                    generated_entities.append(entity)
                    generated_finelabels.append(fine_label)
                    generated_counts[fine_label] += 1

                loop_count += 1


Processing index: 0, label: Antagonist
Processing index: 1, label: Antagonist
Processing index: 2, label: Antagonist
Processing index: 3, label: Antagonist
Processing index: 4, label: Antagonist
Processing index: 5, label: Antagonist
Processing index: 6, label: Antagonist
Processing index: 7, label: Antagonist
Processing index: 8, label: Antagonist
Processing index: 9, label: Protagonist
Processing index: 10, label: Antagonist
Processing index: 11, label: Antagonist
Processing index: 12, label: Antagonist
Processing index: 13, label: Innocent
Processing index: 14, label: Antagonist
Processing index: 15, label: Antagonist
Processing index: 16, label: Antagonist
Processing index: 17, label: Protagonist
Processing index: 18, label: Protagonist
Processing index: 19, label: Protagonist
Processing index: 20, label: Antagonist
Processing index: 21, label: Antagonist
Processing index: 22, label: Antagonist
Processing index: 23, label: Antagonist
Processing index: 24, label: Antagonist
Processi

In [10]:
generated = pd.DataFrame({
    'sentence': generated_sentences,
    'label': generated_labels,
    'mentioned_entity': generated_entities,  # Include mentioned entity
    'fine_grained_role1': generated_finelabels
})

print("Generated Counts:", generated_counts)

generated


Generated Counts: Counter({'Victim': 64, 'Guardian': 34, 'Peacemaker': 24, 'Scapegoat': 12, 'Virtuous': 10, 'Martyr': 10, 'Underdog': 8, 'Forgotten': 4, 'Rebel': 4})


sentence        label  \
0    Ukraine, a victim of Russia's unprovoked and b...     Innocent   
1    Israel, a nation facing complex geopolitical c...     Innocent   
2    Despite the ongoing conflict,  German Chancell...     Innocent   
3    Despite the ongoing conflict and Zelensky's co...     Innocent   
4    Amidst the horrifying scenes of the 2023 Israe...     Innocent   
..                                                 ...          ...   
165  McConnell, acting as a guardian of Ukraine's s...  Protagonist   
166  Dmitry Medvedev,  Deputy Chairman of the Secur...  Protagonist   
167  Dmitry Medvedev's condemnation of the attack a...  Protagonist   
168  Amidst escalating tensions, the Kremlin's resp...  Protagonist   
169  While reinforcements were being flown in from ...  Protagonist   

                  mentioned_entity fine_grained_role1  
0                           Russia             Victim  
1                           Israel             Victim  
2    German Chancellor Olaf Scholz             Victim  
3                          Blinken             Victim  
4    German Chancellor Olaf Scholz             Victim  
..                             ...                ...  
165                        Kremlin           Guardian  
166                Dmitry Medvedev           Guardian  
167            U.S. administration           Guardian  
168            U.S. administration           Guardian  
169              Ukrainian leaders           Guardian  

[170 rows x 4 columns]

In [15]:
generated.to_csv("pleasework4.csv", index=False)

In [12]:
# import pandas as pd(
# import time

# dff = df.copy()
# generated_sentences = []
# generated_labels = []
# generated_entities = []  # To store mentioned entities
# generated_finelabels = []

# for index, row in dff.iterrows():
#     sentence = row['contextual_content']
#     entity = row['entity_mention']
#     label = row['main_role']
#     fine_label = row['fine_grained_role1']
#     print(index)

#     time.sleep(7)

#     if index < 400:
#         count = 0
#         if label == 'Innocent':
#             response = models.generate_content([
#                 f"Generate a sentence based on the context: '{sentence}' with the mentioned entity: '{entity}'. "
#                 f"The sentence should contain the mentioned entity:'{entity}'."
#                 f"The sentence should reflect the '{fine_label}' role perspective. Each sentence should be exactly 500 characters including the entity name. "
#                 f"Do not start with a number or special character."
#             ])

#             if response.parts:
#                 sen = response.text
#                 sen = sen.strip()
#                 generated_sentences.append(sen)
#                 generated_labels.append(label)
#                 generated_entities.append(entity)
#                 generated_finelabels.append(fine_label)
#     else:
#         break


# generated = pd.DataFrame({
#     'sentence': generated_sentences,
#     'label': generated_labels,
#     'mentioned_entity': generated_entities,  # Include mentioned entity
#     'fine_grained_role1':generated_finelabels
# })

# generated


In [29]:
# prompt: read pleasework.csv into df1

df1 = pd.read_csv("pleasework1.csv")
df2 = pd.read_csv("pleasework2.csv")
df3 = pd.read_csv("pleasework3.csv")
df4 = pd.read_csv("pleasework4.csv")
df1

sentence        label  \
0   Faced with Russia's escalating aggression and ...  Protagonist   
1   Amid escalating tensions, NATO's consideration...  Protagonist   
2   Facing immense pressure and a potential catast...     Innocent   
3   China, a nation striving for peaceful developm...     Innocent   
4   Amid escalating tensions,  the innocent people...     Innocent   
..                                                ...          ...   
86  The United States, in its role as an evacuatin...     Innocent   
87  Washington, in its handling of Operation Babyl...     Innocent   
88  While Joe Biden wasn't directly involved in Op...     Innocent   
89  Seymour Hersh, while not directly involved in ...     Innocent   
90  While acknowledging the complexities of the No...  Protagonist   

   mentioned_entity fine_grained_role1  
0            Russia           Guardian  
1             China           Guardian  
2             Putin             Victim  
3             China             Victim  
4       North Korea             Victim  
..              ...                ...  
86    United States          Exploited  
87       Washington          Exploited  
88        Joe Biden          Exploited  
89    Seymour Hersh          Exploited  
90          Ukraine           Virtuous  

[91 rows x 4 columns]

In [30]:
dff = df.copy()
dff = dff.drop(columns=['article_id', 'entity_mention','start_offset', 'end_offset' , 'article_content','index'])
df1 = df1.drop(columns=['mentioned_entity'])
df2 = df2.drop(columns=['mentioned_entity'])
df3 = df3.drop(columns=['mentioned_entity'])
df4 = df4.drop(columns=['mentioned_entity'])
dff = dff.rename(columns={'contextual_content': 'sentence'})
dff = dff.rename(columns={'fine_grained_role1': 'fine_grained_role1'})
dff = dff.rename(columns={'main_role': 'label'})

In [31]:
dff

label fine_grained_role1  \
0     Antagonist                Spy   
1     Antagonist         Instigator   
2     Antagonist          Terrorist   
3     Antagonist        Incompetent   
4     Antagonist        Incompetent   
..           ...                ...   
394   Antagonist         Instigator   
395  Protagonist           Guardian   
396  Protagonist           Guardian   
397   Antagonist          Terrorist   
398   Antagonist          Terrorist   

                                              sentence  
0    r in October. If this rate continues, this yea...  
1    l it completed its trek across many states. In...  
2    umber of injuries and deaths in the war neared...  
3    aff for the Office of the President of Ukraine...  
4    on in an address to the hawkish DC-based Hudso...  
..                                                 ...  
394  Russia Says US Is Responsible for Deadly Ukrai...  
395   injured, including 82 taken to hospitals. Twe...  
396  rs were being flown in from other parts of Rus...  
397  Council, described the incident as “a vile, de...  
398  e incident as “a vile, despicable act against ...  

[399 rows x 3 columns]

In [32]:
df_combined = pd.concat([df1,df2,df3,df4,dff], ignore_index=True)
df_shuffled = df_combined.sample(frac=1).reset_index(drop=True)
df_shuffled.to_csv("pleaseee.csv", index=False)

In [36]:
df_shuffled

sentence        label  \
0    The World Bank, while initially a protagonist ...  Protagonist   
1    ve been thrown into the war in Ukraine," the c...  Protagonist   
2    eatened": Estonian Prime Minister calls for lo...  Protagonist   
3    The Fed, innocent in the eyes of some, a victi...     Innocent   
4    ia should not exist,” he continued. “And Finla...  Protagonist   
..                                                 ...          ...   
757  ial media seized on the clip, including Turnin...     Innocent   
758  Boris Johnson, alongside other world leaders, ...  Protagonist   
759  TH LIVES on at https://sgtreport.tv/\n\nPropag...   Antagonist   
760  orruption go unchallenged.”\n\nThe organizatio...   Antagonist   
761  Boris Johnson, acting as a guardian of Ukraine...  Protagonist   

    fine_grained_role1  
0           Peacemaker  
1           Peacemaker  
2             Underdog  
3               Victim  
4             Guardian  
..                 ...  
757             Victim  
758           Guardian  
759           Deceiver  
760          Terrorist  
761           Guardian  

[762 rows x 3 columns]

In [41]:
import google.generativeai as genai
from google.colab import userdata
import pandas as pd

from IPython.display import display
from IPython.display import Markdown

GOOGLE_AI_STUDIO = userdata.get('gggg')

genai.configure(api_key=GOOGLE_AI_STUDIO)

In [43]:
!pip -q install google-generativeai==0.3.0
!pip -q install google-ai-generativelanguage==0.4.0

for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models = genai.GenerativeModel('gemini-1.5-flash-002')

ReadTimeout: HTTPConnectionPool(host='localhost', port=41677): Read timed out. (read timeout=60.0)

In [ ]:
import pandas as pd
import time
from collections import Counter

# List of tuples for fine labels and counts
roles = [
    ("Protagonist", 200), ("Innocent", 200), ("Innocent", 200)
]

# Converting roles list to a dictionary for checking counts
roles_dict = dict(roles)

# To track generated counts for each fine_label
generated_counts = Counter()


dff = df_shuffled.copy()
generated_sentences = []
generated_labels = []
generated_entities = []  # To store mentioned entities
generated_finelabels = []

for index, row in dff.iterrows():
    sentence = row['sentence']
    fine_label = row['fine_grained_role1']
    label = row['label']
    print(f"Processing index: {index}, label: {label}")

    if generated_counts[label] >= 400:
            continue
    if index < 700:
        if label in roles_dict:
            loop_count = 0
            while generated_counts[fine_label] < 400 and loop_count < 2:

                # Pick an unused entity name
                if loop_count == 0:
                    ent_val = 0
                elif loop_count == 1:
                    ent_val = 1
                entity = all_entities[index+ent_val]

                time.sleep(7)  # Delay to respect generation constraints

                response = models.generate_content([
                    f"Generate a sentence based on the context: '{sentence}' using the mentioned entity: '{entity}'. "
                    f"The sentence should contain the mentioned entity:'{entity}'. "
                    f"The sentence should reflect the '{label}' and '{fine_label}' role perspective with respect to the '{entity}'. Each sentence should be exactly 500 characters including the entity name. "
                    f"Do not start with a number or special character."
                ])

                if response.parts:
                    sen = response.text.strip()
                    generated_sentences.append(sen)
                    generated_labels.append(label)
                    generated_entities.append(entity)
                    generated_finelabels.append(fine_label)
                    generated_counts[fine_label] += 1

                loop_count += 1




In [ ]:
generated = pd.DataFrame({
    'sentence': generated_sentences,
    'label': generated_labels,
    'mentioned_entity': generated_entities,  # Include mentioned entity
    'fine_grained_role1': generated_finelabels
})

print("Generated Counts:", generated_counts)

generated


In [ ]:
df_combined2 = pd.concat([df_shuffled, generated], ignore_index=True)
df_shuffled2 = df_combined2.sample(frac=1).reset_index(drop=True)
df_combined2.to_csv("pleaseeeee2.csv", index=False)
# import pandas as pd


In [14]:
output_file = '/content/dataset/big_data.csv'
result_df.to_csv(output_file, index=False)

NameError: name 'result_df' is not defined

In [ ]:
# result_df = result_df.drop('File Content', axis=1)
# result_df = result_df.drop('File Name', axis=1)
# result_df = result_df.drop('Protagonist_', axis=1)
# result_df = result_df.drop('Innocent_', axis=1)
# result_df = result_df.drop('Antagonist_', axis=1)
traindf = result_df.sample(frac=1, random_state=42).reset_index(drop=True)
# traindf = result_df.copy()
train_data = traindf.iloc[:240]
# train_data = train_data.sample(frac=1, random_state=42).reset_index(drop=True)
validation_data = traindf.iloc[240:270]
test_data = traindf.iloc[270:]
# print(train_data)
label_df = pd.DataFrame()
# labelq_df = pd.DataFrame()
# labelq_df['File Name'] = traindf['File Name']
# print(labelq_df)

In [ ]:
# result_df = result_df.sample(frac=1, random_state=22).reset_index(drop=True)
# result_df

In [ ]:
# kk_index = traindf.columns.get_loc('Mentioned Entity')
kk_index = traindf.columns.get_loc('label')
label_df = traindf.iloc[:, kk_index + 1:]

# combined_df = pd.concat([labelq_df, label_df], axis=1)
label_df.fillna(0, inplace=True)
label_df

In [ ]:
train_labe = label_df.iloc[:240]
# train_labe = train_labe.sample(frac=1, random_state=42).reset_index(drop=True)
validation_labe = label_df.iloc[240:270]
test_labels = label_df.iloc[270:]
train_labe

In [ ]:
# result = train_data['Protagonist'].equals(train_labe['Protagonist'])
# print(result)

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast, BertModel
from torch.utils.data import Dataset
import torch

checkpoint = "microsoft/deberta-v3-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
model

In [ ]:
# output_file = '/content/dataset/final_data.csv'
# result_df.to_csv(output_file, index=False)

In [ ]:
# class ArgDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# train_sentences = train_data['Contextual Content'].values.tolist()
# train_labels = train_labe.iloc[:, 0:].values
# train_labels = train_labels.astype(np.float32)

# validation_sentences = validation_data['Contextual Content'].values.tolist()
# validation_labels = validation_labe.iloc[:, 0:].values
# validation_labels = validation_labels.astype(np.float32)

# train_encodings = tokenizer(train_sentences, padding = True, truncation = True, max_length = 512)
# train_dataset = ArgDataset(train_encodings, train_labels)

# val_encodings = tokenizer(validation_sentences, padding = True, truncation = True, max_length = 512)
# validation_dataset = ArgDataset(val_encodings, validation_labels)

In [ ]:
# from datasets import Dataset

# df_1 = train_data
# df_1['label'] = train_labe.iloc[:, 0:].values.astype(np.float32).tolist()
# df_train = df_1[['Contextual Content', 'label']].copy()

# df_1 = validation_data
# df_1['label'] = validation_labe.iloc[:, 0:].values.astype(np.float32).tolist()
# df_val = df_1[['Contextual Content', 'label']].copy()

# def tokenize_function(example):
#   return tokenizer(example['Contextual Content'],truncation=True, padding = True)

# traindataset = Dataset.from_pandas(df_train)
# train_dataset = traindataset.map(tokenize_function,batched=True)

# validationdataset = Dataset.from_pandas(df_val)
# validation_dataset = validationdataset.map(tokenize_function,batched=True)

In [ ]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions > 0.5
#     accuracy = accuracy_score(labels, preds)
#     precision = precision_score(labels, preds, average='weighted')
#     recall = recall_score(labels, preds, average='weighted')
#     f1 = f1_score(labels, preds, average='weighted')

#     return {
#         'accuracy': accuracy,
#         'precision': precision,
#         'recall': recall,
#         'f1': f1
#     }

# # label_names
train_labe

In [ ]:
from datasets import Dataset
import numpy as np

df_1 = train_data.copy()
df_1['labe'] = train_labe.iloc[:, 0:].values.astype(np.float32).tolist()
df_1['Prompt'] = "Given the context: " + df_1['Contextual Content'] + ", decide the label for the mentioned entity: " + df_1['Mentioned Entity'] + "."

df_train = df_1[['Prompt', 'labe']].copy()

df_1 = validation_data.copy()
df_1['labe'] = validation_labe.iloc[:, 0:].values.astype(np.float32).tolist()
df_1['Prompt'] = "Given the context: " + df_1['Contextual Content'] + ", decide the label for the mentioned entity: " + df_1['Mentioned Entity'] + "."

df_val = df_1[['Prompt', 'labe']].copy()

def tokenize_function(example):
    return tokenizer(example['Prompt'], truncation=True, padding=True)

traindataset = Dataset.from_pandas(df_train)
train_dataset = traindataset.map(tokenize_function, batched=True)

validationdataset = Dataset.from_pandas(df_val)
validation_dataset = validationdataset.map(tokenize_function, batched=True)

df_train['Prompt']

In [ ]:
df = pd.DataFrame(train_dataset)

# Print the first few rows
print(df['Prompt'])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# label_names = train_labels_df.columns[1:].tolist()
# label_names = ["Protagonist",	"Antagonist",	"Innocent"]


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions > 0.5
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    # report = classification_report(labels, preds, target_names=label_names)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        # 'classification_report': report
    }

# def compute_loss(model, inputs, return_outputs=False):
#     """
#     Computes the loss for the model.

#     Args:
#         model: The model to compute the loss for.
#         inputs: The inputs to the model.
#         return_outputs: Whether to return the model outputs along with the loss.

#     Returns:
#         The loss value, or a tuple of (loss, outputs) if return_outputs is True.
#     """
#     from torch.nn import BCEWithLogitsLoss # Import the appropriate loss function

#     labels = inputs.get("labels") # Assuming your labels are in the 'labels' key
#     outputs = model(**inputs)
#     logits = outputs.logits
#     loss_fn = BCEWithLogitsLoss() # Initialize your loss function
#     loss = loss_fn(logits.view(-1), labels.float().view(-1)) # Calculate the loss

#     return (loss, outputs) if return_outputs else loss

# label_names

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=".",
    eval_strategy="epoch",
    logging_dir="./logs",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=500,
    metric_for_best_model="f1",
    # prediction_loss_only =True
)

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
 )

trainer.train()


In [ ]:
model.save_pretrained("/content/")
model = AutoModelForSequenceClassification.from_pretrained("/content/")

In [ ]:
test_labels.loc[:, 'Argument ID'] = range(1, len(test_labels) + 1)
test_labels

In [ ]:
test_data.loc[:, 'Argument ID'] = range(1, len(test_data) + 1)
test_data

In [ ]:

!mkdir -p /content/output


In [ ]:
predictions = []
threshold = 0.5

for idx, row in test_data.iterrows():
    arg_id = row['Argument ID']
    text = row['Contextual Content']
    encoding = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**encoding)

    sigmoid = torch.nn.Sigmoid()
    predictions_tensor = sigmoid(outputs.logits)

    predicted_labels = []
    for prob in predictions_tensor.flatten():
        if prob >= threshold:
          label = 1
        else:
          label =  0
        predicted_labels.append(label)

    predictions.append([arg_id] + predicted_labels)

data = {'Argument ID': [entry[0] for entry in predictions]}

for i, label in enumerate(test_labels.columns[:-1]):
    data[label] = [entry[i+1] for entry in predictions]
predictions_df = pd.DataFrame(data)

In [ ]:
output_path = "/content/output/my_model_predictions.tsv"
predictions_df.to_csv(output_path, sep='\t', index=False)
output_path = "/content/output/true_labels.tsv"
test_labels.to_csv(output_path, sep='\t', index=False)

In [ ]:
has_nonzero = (predictions_df.iloc[:, 1:] != 0).any().any()
print(has_nonzero)

In [ ]:
predictions_df

In [ ]:
test_labels